In [1]:
import os

In [2]:
%pwd

'/Users/alamgirkabir/chicken-diease-classification/experiment'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/alamgirkabir/chicken-diease-classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from chickenDieaseClassifier.constants import *
from chickenDieaseClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config



In [8]:
import os
import urllib.request as request
import zipfile
import gdown
from chickenDieaseClassifier import logger
from chickenDieaseClassifier.utils.common import get_size

/Users/alamgirkabir/chicken-diease-classification/chicken-diease/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [24]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
    
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                file_id = self.config.source_url.split('/')[-2]
                gdown.download(url='https://drive.google.com/uc?id='+file_id, output=self.config.local_data_file, quiet=False)
                logger.info(f"Data downloaded at {self.config.local_data_file}")
            except Exception as e:
                raise e

        else:
            logger.info(f"File already exist od size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """Extracts the zip file into the data directory.
            The function return none
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [25]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-04 11:40:36,792: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-04 11:40:36,801: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-04 11:40:36,815: INFO: common: created directory at: artifacts]
[2024-01-04 11:40:36,820: INFO: common: created directory at: artifacts/data_ingestion]


Downloading...
From (uriginal): https://drive.google.com/uc?id=168s9bZXSFPFrmJwkyr_Z86WYGyl20aoE
From (redirected): https://drive.google.com/uc?id=168s9bZXSFPFrmJwkyr_Z86WYGyl20aoE&confirm=t&uuid=a4c1e341-d791-4c74-b667-abad135f3f13
To: /Users/alamgirkabir/chicken-diease-classification/artifacts/data_ingestion/data.zip
100%|██████████| 279M/279M [00:12<00:00, 21.9MB/s] 


[2024-01-04 11:40:50,781: INFO: 4213019196: Data downloaded at artifacts/data_ingestion/data.zip]
